In [1]:
import pandas as pd
import numpy as np
import datetime


In [2]:
students = pd.read_csv("Data/Students data/students_processed_2.csv")
batla = pd.read_csv("Output/BONUS_HASAM_202206233_35.csv")

In [3]:
students["end_date"] = students["end_date"].apply(pd.to_datetime, format='%Y-%m-%d')

In [4]:
## keep columns
keep_cols = ["id","gender","end_date","end_year","mosad_id","eshcol","group"]
students = students[keep_cols]

In [5]:
## drop Unnamed: 0
batla = batla.drop(columns=["Unnamed: 0"])

In [6]:
## Merge Leftovers
merged = pd.merge(students, batla, how="outer", on="id", indicator=True)
left_only = merged[merged["_merge"] == "left_only"]

In [12]:
left_only["end_year"].value_counts(sort=True)

2011.0    90
2012.0    80
2015.0    72
2013.0    68
2019.0    66
2016.0    65
2018.0    52
2014.0    46
2017.0    45
2020.0    39
2010.0    34
1900.0    28
2021.0    25
Name: end_year, dtype: int64

In [7]:
df = pd.merge(batla, students, on="id", how="left")

In [10]:
nehiga = students.loc[(students["eshcol"] == 'נהיגה ')]
nehiga = nehiga.loc[(nehiga["end_year"] == 2018) | (nehiga["end_year"] == 2019)]
nehiga["id"].nunique()

16898

In [7]:
## Filter after 2016
df = df.loc[(df["end_year"] >= 2016)]

In [10]:
df["date"] =  pd.to_datetime(df["date"])
#df["end_date"] =  pd.to_datetime(df["end_date"])

def inter(x):

    if x["date"].year == x["end_date"].year and x["date"].month == x["end_date"].month:
        return 1
    else:
        return 0

df["intervention"] = df.apply(inter, axis=1)

In [11]:
# Index monthes before and after intervention
df = df.sort_values(by=['id','date'], ascending=False)
df['intervention_index_pre'] = df.groupby('id',group_keys=False)['intervention'].apply(lambda x: (x - x.cumsum()).where(x == 1, lambda x: x.cumsum()))

df = df.sort_values(by=['id','date'], ascending=True)
df['intervention_index_post'] = df.groupby('id',group_keys=False)['intervention'].apply(lambda x: (x - x.cumsum()).where(x == 1, lambda x: x.cumsum()))
df["intervention_index_post"] = df["intervention_index_post"]*-1

def intervention(x):
    if x["intervention_index_pre"] < 0:
        return x["intervention_index_pre"]
    if x["intervention_index_pre"] == 0:
        return x["intervention_index_post"]
    else:
        return 0

df["intervention_index"] = df.apply(intervention, axis=1)
df= df.drop(columns=["intervention_index_pre","intervention_index_post"])

In [12]:
## Drop observation finished after batla data range
df = df.loc[(df["intervention"]!=0) | (df["intervention_index"]!=0)]

In [14]:
df.to_csv("Output/merge_stud_batla.csv", index=False)